In [1]:
import random as r
import json

from tensorflow import keras
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from AbstractArchitecture_v2 import AbstractArchitecture
from DenseEncoder import DenseEncoder
from DenseDecoder import DenseDecoder
from NormalizedMeanSquaredError import NormalizedMeanSquaredError as NMSE
from plot_model_prediction import plot_model_prediction

In [2]:
# Set data file locations
data_file_prefix = './data/NLSL_expt1'

# Step 1. Load in the data
data_train_u = np.load("{}_train1_u.npy".format(data_file_prefix)).astype(np.float32)
data_train_f = np.load("{}_train1_f.npy".format(data_file_prefix)).astype(np.float32)
data_val_u = np.load("{}_val_u.npy".format(data_file_prefix)).astype(np.float32)
data_val_f = np.load("{}_val_f.npy".format(data_file_prefix)).astype(np.float32)
data_test_u1 = np.load("{}_test1_u.npy".format(data_file_prefix)).astype(np.float32)
data_test_f1 = np.load("{}_test1_f.npy".format(data_file_prefix)).astype(np.float32)
data_test_u = np.load("{}_test2_u.npy".format(data_file_prefix)).astype(np.float32)
data_test_f = np.load("{}_test2_f.npy".format(data_file_prefix)).astype(np.float32)


In [3]:
best_model_path = "./model_weights/NLSL_Experiment_06a/final_model"

# Load the best model
full_model = keras.models.load_model(best_model_path, 
                                     custom_objects={"NormalizedMeanSquaredError": NMSE})


In [6]:
loss_fns = 4*[NMSE()]
predicted_ys = full_model.predict(x=[data_train_u, data_train_f])

true_ys=[data_train_u, data_train_f, data_train_f, data_train_u]
nmse = NMSE()

total_loss = 0
for i, (pred_y, true_y, loss_fn) in enumerate(zip(predicted_ys, true_ys, loss_fns)):
    #print(pred_y.dtype, true_y.dtype)
    print("Score {}:".format(i+1), end="  ")
    loss = loss_fn(pred_y, true_y)
    tf.print(loss)
    total_loss += loss
    
print(data_train_u.shape)
print("Total Loss:", total_loss)

Score 1:  1.392583e-05
Score 2:  5.33852035e-05
Score 3:  0.00028553803
Score 4:  3.13317432e-05
(8906, 128)
Total Loss: tf.Tensor(0.0003841808, shape=(), dtype=float32)


In [7]:
val_x = [data_val_u, data_val_f]
val_ys = [data_val_u, data_val_f, data_val_f, data_val_u]
predicted_ys = full_model.predict(x=val_x)

total_loss = 0
for i, (pred_y, true_y, loss_fn) in enumerate(zip(predicted_ys, val_ys, loss_fns)):
    #print(pred_y.dtype, true_y.dtype)
    print("Score {}:".format(i), end="  ")
    loss = loss_fn(pred_y, true_y)
    tf.print(loss)
    total_loss += loss
    
print(data_train_u.shape)
print("Total Loss:", total_loss)

Score 0:  1.63723307e-05
Score 1:  7.97558314e-05
Score 2:  0.000870037125
Score 3:  0.000135763723
(8906, 128)
Total Loss: tf.Tensor(0.001101929, shape=(), dtype=float32)
